<a href="https://colab.research.google.com/github/vasu-rguktn/Team_X/blob/main/model_1_using_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas numpy scikit-learn lightgbm kaggle


Unzip the dataset

In [2]:
!unzip archive.zip


Archive:  archive.zip
replace building_metadata.feather? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

extracted files

In [3]:
!ls


archive.zip		   sample_submission.feather  weather_test.feather
building_metadata.feather  test.feather		      weather_train.feather
sample_data		   train.feather


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install pyarrow  #mandatory to load dataset




In [5]:
!pwd
!ls


/content
archive.zip		   sample_submission.feather  weather_test.feather
building_metadata.feather  test.feather		      weather_train.feather
sample_data		   train.feather


Load data set

In [6]:
train = pd.read_feather("/content/train.feather")
building = pd.read_feather("/content/building_metadata.feather")
weather = pd.read_feather("/content/weather_train.feather")



In [7]:
train.shape, building.shape, weather.shape


((20216100, 4), (1449, 6), (139773, 9))

merge data sets

In [8]:
df = train.merge(building, on="building_id", how="left")
df = df.merge(weather, on=["site_id", "timestamp"], how="left")
df.head()



,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0


Handle missing values

In [9]:
df.isnull().sum()



,0
building_id,0
meter,0
timestamp,0
meter_reading,0
site_id,0
primary_use,0
square_feet,0
year_built,12127645
floor_count,16709167
air_temperature,96658


In [10]:
df.fillna(method="ffill", inplace=True)


/tmp/ipython-input-3651561093.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


Encode categorical values

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["primary_use"] = le.fit_transform(df["primary_use"])


In [14]:
df_sample = df.sample(n=100000, random_state=42)

Feature selection (X) & Target (y)

In [17]:
X = df_sample.drop(["meter_reading", "timestamp"], axis=1)
y = df_sample["meter_reading"]


split the data for training and testing

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Train Random Forest Model

In [19]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

In [24]:
y_pred = rf.predict(X_test)






In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("MAE:", mae)
print("RMSE:", rmse)
print("R2 Score:", r2)

MAE: 2420.5797208470826
RMSE: 128069.73923180866
R2 Score: 0.48230016960375466
